# Instalación de dependencias

In [ ]:
pip install kagglehub

In [ ]:
pip install --upgrade pandas

# Descarga del dataset

In [ ]:
import os
import kagglehub

dataset_path = kagglehub.dataset_download("romanniki/food-delivery-cost-and-profitability")

# Mostrar todos los archivos encontrados
for root, dirs, files in os.walk(dataset_path):
    for file in files:
        print("Archivo encontrado:", os.path.join(root, file))

In [ ]:
import kagglehub
import os

# Descarga el dataset (o usa el caché)
romanniki_food_delivery_cost_and_profitability_path = kagglehub.dataset_download('romanniki/food-delivery-cost-and-profitability')

# Buscar el primer archivo dentro del dataset descargado
for root, dirs, files in os.walk(romanniki_food_delivery_cost_and_profitability_path):
    for file in files:
        path_dataset = os.path.abspath(os.path.join(root, file))
        break  # solo el primero
    else:
        continue
    break

print('Data source import complete.')
print("Archivo encontrado en:", path_dataset)

# Librerías necesarias

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

# Exploración Inicial (EDA)

In [ ]:
df = pd.read_csv(path_dataset)
df.shape

In [ ]:

# Convertir fechas a tipo datetime
df['Order Date and Time'] = pd.to_datetime(df['Order Date and Time'])
df['Delivery Date and Time'] = pd.to_datetime(df['Delivery Date and Time'])

# Crear columna de tiempo de entrega en minutos
df['Delivery Duration (min)'] = (df['Delivery Date and Time'] - df['Order Date and Time']).dt.total_seconds() / 60
# Extraer día y hora del pedido
df['day_of_week'] = df['Order Date and Time'].dt.day_name()
df['hour_of_day'] = df['Order Date and Time'].dt.hour
# Marcar si hay descuento o no
df['has_discount'] = df['Discounts and Offers'].str.lower() != 'none'

# Marcar si fue reembolsado
df['refunded'] = df['Refunds/Chargebacks'] > 0
df

In [ ]:
print(df['Discounts and Offers'].value_counts())

### Cálculo del valor del descuento

In [ ]:
def calcular_descuento(row):
    valor = str(row['Discounts and Offers']).lower()
    pedido = row['Order Value']
    
    # Si es porcentaje, por ejemplo "10%" o "5% on app"
    if '%' in valor:
        try:
            porcentaje = float(valor.split('%')[0])
            return pedido * (porcentaje / 100)
        except:
            return 0
        
    # Si es "50 off Promo" u otros con valor fijo
    elif 'off' in valor:
        for token in valor.split():
            try:
                return float(token)
            except:
                continue
        return 0

    # Si es "None" u otra cosa no reconocida
    return 0

# Aplicar al DataFrame
df['Discount Value'] = df.apply(calcular_descuento, axis=1)

# Verificar
df[['Discounts and Offers', 'Order Value', 'Discount Value']].head(10)

### Cálculo del coste total y beneficio

In [ ]:
df['Total Cost'] = df['Delivery Fee'] + df['Discount Value'] + df['Payment Processing Fee'] + df['Refunds/Chargebacks']

In [ ]:
df['Profit'] = df['Commission Fee'] - df['Total Cost']

# Estadísticas generales
print("Ganancia media por pedido:", round(df['Profit'].mean(), 2))
print("Pedidos con pérdida:", (df['Profit'] < 0).sum())
print("Porcentaje con pérdida:", round((df['Profit'] < 0).mean() * 100, 2), '%')

### Análisis de pérdidas

In [ ]:
# Mostrar los 10 pedidos con mayores pérdidas (menor Profit)
top_losses = df.sort_values(by='Profit').tail(10)

# Mostrar solo las columnas clave para el análisis
top_losses[[ 'Order Value', 'Commission Fee', 'Total Cost', 'Profit', 'Discounts and Offers']]

In [ ]:
print(df)

## Visualización de correlaciones numéricas

In [ ]:
plt.figure(figsize=(6,6))
sns.heatmap(df.select_dtypes(include=['number']).corr(), annot=True, fmt='.1f')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10,5))
sns.boxplot(data=df, x='day_of_week', y='Delivery Duration (min)', 
            order=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])
plt.xticks(rotation=45)
plt.title("Duración de entrega por día de la semana")
plt.show()

In [ ]:
top_restaurants = df['Restaurant ID'].value_counts().nlargest(10).index

plt.figure(figsize=(10,5))
sns.boxplot(data=df[df['Restaurant ID'].isin(top_restaurants)],
            x='Restaurant ID', y='Delivery Duration (min)')
plt.xticks(rotation=45)
plt.title("Top 10 restaurantes vs duración de entrega")
plt.show()

In [ ]:
plt.figure(figsize=(6,4))
sns.boxplot(data=df, x='has_discount', y='Delivery Duration (min)')
plt.title("Duración con/sin descuento")
plt.xlabel("¿Tiene descuento?")
plt.ylabel("Duración (min)")
plt.show()

In [ ]:
plt.figure(figsize=(8,4))
sns.boxplot(data=df, x='Payment Method', y='Delivery Duration (min)')
plt.title("Método de pago vs duración de entrega")
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(6,4))
sns.boxplot(data=df, x='refunded', y='Delivery Duration (min)')
plt.title("Reembolsos vs duración de entrega")
plt.xlabel("¿Fue reembolsado?")
plt.ylabel("Duración (min)")
plt.show()

In [ ]:
plt.figure(figsize=(10,4))
sns.boxplot(data=df, x='hour_of_day', y='Delivery Duration (min)')
plt.title("Duración según hora del pedido")
plt.xlabel("Hora del día")
plt.ylabel("Duración (min)")
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Asegurarnos de que la columna tenga un nombre limpio (por si no lo hiciste antes)
df['Discounts and Offers'] = df['Discounts and Offers'].astype(str)

# Agrupar por tipo de descuento y calcular profit promedio
profit_por_descuento = df.groupby('Discounts and Offers')['Profit'].mean().sort_values()

# Crear gráfico
plt.figure(figsize=(10,6))
sns.barplot(x=profit_por_descuento.values, y=profit_por_descuento.index, palette='coolwarm')
plt.title("Ganancia media por tipo de descuento")
plt.xlabel("Ganancia media (Profit)")
plt.ylabel("Tipo de descuento")
plt.axvline(0, color='gray', linestyle='--')
plt.tight_layout()
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Asegúrate de que la columna de duración esté creada
if 'Delivery Duration (min)' not in df.columns:
    df['Order Date and Time'] = pd.to_datetime(df['Order Date and Time'])
    df['Delivery Date and Time'] = pd.to_datetime(df['Delivery Date and Time'])
    df['Delivery Duration (min)'] = (df['Delivery Date and Time'] - df['Order Date and Time']).dt.total_seconds() / 60

# Crear columna binaria: ¿Hubo reembolso?
df['Refunded'] = df['Refunds/Chargebacks'] > 0

# Gráfico de distribución del tiempo de entrega según haya o no reembolso
plt.figure(figsize=(10,6))
sns.boxplot(x='Refunded', y='Delivery Duration (min)', data=df)
plt.xticks([0,1], ['Sin reembolso', 'Con reembolso'])
plt.title("Tiempo de entrega según presencia de reembolso")
plt.ylabel("Duración de entrega (minutos)")
plt.xlabel("¿Hubo reembolso?")
plt.tight_layout()
plt.show()

In [ ]:
# Normalizar y deduplicar nombres de columnas en pandas (compatible 1.x/2.x)
from collections import defaultdict
df.columns = (
    df.columns
      .map(str)
      .str.strip()
      .str.lower()
      .str.replace(' ', '_', regex=False)
)
counts = defaultdict(int)
new_cols = []
for c in df.columns:
    if counts[c]:
        new_cols.append(f"{c}.{counts[c]}")
    else:
        new_cols.append(c)
    counts[c] += 1
df.columns = new_cols


In [ ]:
print (df)

# Configuración de Spark y preparación de datos

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.regression import GBTRegressor
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit

# Cerrar contexto anterior si está activo
if SparkContext._active_spark_context:
    SparkContext._active_spark_context.stop()

# SparkSession apuntando al cluster
spark = (
    SparkSession.builder.appName("DeliveryDurationGBT-Tuned-Light")
    .master("spark://agile:7077")
    .config("spark.driver.bindAddress", "0.0.0.0")
    .getOrCreate()
)

sc = spark.sparkContext
sc.setLogLevel("ERROR")

# Añadir columnas derivadas en pandas antes de pasar a Spark
df["es_fin_de_semana"] = df["day_of_week"].isin(["Saturday", "Sunday"]).astype(int)
df["es_hora_punta"] = df["hour_of_day"].between(13, 15) | df["hour_of_day"].between(20, 22)
df["es_hora_punta"] = df["es_hora_punta"].astype(int)

# Crear DataFrame en Spark
df_spark = spark.createDataFrame(df)

# Variables
categorical_cols = ['payment_method', 'discounts_and_offers', 'day_of_week', 'hour_of_day']
numeric_cols = [
    'order_value', 'delivery_fee', 'commission_fee', 'payment_processing_fee',
    'refunds/chargebacks', 'discount_value', 'has_discount', 'refunded',
    'es_fin_de_semana', 'es_hora_punta'
]
target = 'delivery_duration_(min)'

# Indexado y one-hot
indexers = [StringIndexer(inputCol=col, outputCol=col + "_idx", handleInvalid="keep") for col in categorical_cols]
encoders = [OneHotEncoder(inputCol=col + "_idx", outputCol=col + "_vec") for col in categorical_cols]

# Vector de features
assembler = VectorAssembler(
    inputCols=numeric_cols + [col + "_vec" for col in categorical_cols],
    outputCol="features"
)

# Modelo
regressor = GBTRegressor(featuresCol="features", labelCol=target)

# Pipeline
pipeline = Pipeline(stages=indexers + encoders + [assembler, regressor])

# Grid de hiperparámetros ligera
paramGrid = ParamGridBuilder() \
    .addGrid(regressor.maxDepth, [5]) \
    .addGrid(regressor.maxIter, [20, 40]) \
    .build()

# Train-validation split
tvs = TrainValidationSplit(
    estimator=pipeline,
    estimatorParamMaps=paramGrid,
    evaluator=RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="rmse"),
    trainRatio=0.8
)

# Split de datos
train_data, test_data = df_spark.randomSplit([0.8, 0.2], seed=42)

# Entrenamiento
# tvs_model = tvs.fit(train_data)
tvs_model = pipeline.fit(train_data)

# Evaluación
predictions = tvs_model.transform(test_data)
rmse = RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="rmse").evaluate(predictions)

print("✅ Modelo mejorado con TrainValidationSplit")
print(f"📉 RMSE final: {rmse:.2f}")

In [ ]:
df_spark.printSchema()

### Evaluación del modelo

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Convertimos a Pandas para visualización
preds_pd = predictions.select("delivery_duration_(min)", "prediction").toPandas()

# Plot real vs predicho
plt.figure(figsize=(6,6))
plt.scatter(preds_pd["delivery_duration_(min)"], preds_pd["prediction"], alpha=0.5)
plt.plot([0, preds_pd.max().max()], [0, preds_pd.max().max()], 'r--')
plt.xlabel("Valor real")
plt.ylabel("Predicción")
plt.title("Predicción vs Real (Duración en minutos)")
plt.grid(True)
plt.show()

In [ ]:
# Mostrar predicciones reales vs. predichas
predictions.select('delivery_duration_(min)', 'prediction').show(10)

### Guardado del modelo y transformadores

In [ ]:
import os
base_path = "./models"
os.makedirs(base_path, exist_ok=True)

# Guarda el VectorAssembler
assembler.write().overwrite().save(f"{base_path}/vector_assembler.bin")

# Guarda todos los StringIndexer
for idx, indexer_model in enumerate(indexers):
    indexer_model.write().overwrite().save(f"{base_path}/string_indexer_{idx}.bin")

# Guarda todos los OneHotEncoder
for idx, encoder_model in enumerate(encoders):
    encoder_model.write().overwrite().save(f"{base_path}/one_hot_encoder_{idx}.bin")

# Guarda el modelo entrenado
tvs_model.write().overwrite().save(f"{base_path}/pipeline_model.bin")